In [ ]:
from sys import exit
import numpy as np
import pandas as pd
from os.path import dirname, join, isfile, isdir
from os import makedirs
from datetime import datetime, timedelta

from bulletin.data.tb_pacientes import TbPacientes
from bulletin.data.casos_confirmados import CasosConfirmados
from bulletin.commom.utils import Timer

from hashlib import sha256
from bulletin.commom.normalize import normalize_text, normalize_number, normalize_hash, normalize_cpf

pd.set_option('display.max_columns', None)

output = join("output","correcoes","tb_pacientes")

if not isdir(output):
    makedirs(output)

timer = Timer()

In [ ]:
timer_all = Timer()
timer_all.reset()

In [ ]:
timer.reset()
tb = TbPacientes()
tb.update()
timer.stop()

In [ ]:
timer.reset()
casos_confirmados = CasosConfirmados()
casos_confirmados.update()
timer.stop()

In [ ]:
casost = tb.get()
casosc = casos_confirmados.get_casos()
print(f"casos casos_confirmados: {casosc.shape}")
print(f"casos tb_pacientes: {casost.shape}")

In [ ]:
idx_casos_confirmados = casosc[casosc['hash'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_less'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_more'].isin(casost['hash'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_less_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados += casosc[casosc['hash_more_atend'].isin(casost['hash_atend'])].index.tolist()
idx_casos_confirmados = set(idx_casos_confirmados)
casos_confirmados_nao_tb_pacientes = casosc.loc[ set(casosc.index.tolist()) - idx_casos_confirmados ].sort_values('ordem')

In [ ]:
idx_tb_pacientes = casost[casost['hash'].isin(casosc['hash'])].index.tolist()
idx_tb_pacientes += casost[casost['hash'].isin(casosc['hash_less'])].index.tolist()
idx_tb_pacientes += casost[casost['hash'].isin(casosc['hash_more'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_atend'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_less_atend'])].index.tolist()
idx_tb_pacientes += casost[casost['hash_atend'].isin(casosc['hash_more_atend'])].index.tolist()
idx_tb_pacientes = set(idx_tb_pacientes)
tb_pacientes_nao_casos_confirmados = casost.loc[ set(casost.index.tolist()) - idx_tb_pacientes ].sort_values('identificacao')


In [ ]:
print(f"casos_confirmados_nao_tb_pacientes: {casos_confirmados_nao_tb_pacientes.shape}")
if len(casos_confirmados_nao_tb_pacientes) > 0:
    casos_confirmados_nao_tb_pacientes.sort_values('nome').to_excel(join(output,'casos_confirmados_nao_tb_pacientes.xlsx'), index=False)
print(f"tb_pacientes_nao_casos_confirmados: {tb_pacientes_nao_casos_confirmados.shape}")
if len(tb_pacientes_nao_casos_confirmados) > 0:
    tb_pacientes_nao_casos_confirmados.sort_values('nome').to_excel(join(output,'tb_pacientes_nao_casos_confirmados.xlsx'), index=False)

In [ ]:
casost['ibge_res_pr'] = casost['ibge_res_pr'].fillna('9999999')
casosc['ibge7'] = casosc['ibge7'].fillna('9999999')

In [ ]:
casosc['nome_hash'] = casosc['nome'].apply(normalize_hash)
casost['nome_hash'] = casost['nome'].apply(normalize_hash)

In [ ]:
print(all(casosc['nome_hash'].values == casost['nome_hash'].values))
print(all(casosc['ibge7'].values == casost['ibge_res_pr'].values))
# exit()

In [ ]:
timer.reset()

sorted_casosc = pd.DataFrame(columns=casosc.columns, dtype=casosc.dtypes.values)
for mun, df_mun in casosc.groupby('ibge7'):
    sorted_casosc = sorted_casosc.append(df_mun.sort_values('nome_hash'))

sorted_casost = pd.DataFrame(columns=casost.columns, dtype=casost.dtypes.values)
for mun, df_mun in casost.groupby('ibge_res_pr'):
    sorted_casost = sorted_casost.append(df_mun.sort_values('nome_hash'))

casosc = sorted_casosc
print(f"casos casos_confirmados: {sorted_casosc.shape}")
casost = sorted_casost
print(f"casos tb_pacientes: {casost.shape}")
print(timer.stop())

In [ ]:
print(all(casosc['nome_hash'].values == casost['nome_hash'].values))
print(all(casosc['ibge7'].values == casost['ibge_res_pr'].values))

In [ ]:
casosc = casosc.drop(columns=['hash','hash_less','hash_more','hash_atend','hash_less_atend','hash_more_atend'])
casost = casost.drop(columns=['hash','hash_less','hash_more','hash_atend','hash_less_atend','hash_more_atend'])

In [ ]:
casosc['idade_mais1'] = casosc['idade'] + 1
casosc['idade_menos1'] = casosc['idade'] - 1

In [ ]:
writer = pd.ExcelWriter(join(output,"novos_tb_pacientes.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

casost.to_excel(writer,index=None)

writer.save()
writer.close()

In [ ]:
writer = pd.ExcelWriter(join(output,"casos_ordenados.xlsx"),
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')

casosc.to_excel(writer,index=None)

writer.save()
writer.close()

In [ ]:
print(timedelta(seconds=timer_all.stop()))
exit()

In [30]:

writer = pd.ExcelWriter("obitos_tb.xlsx",
                        engine='xlsxwriter',
                        datetime_format='dd/mm/yyyy',
                        date_format='dd/mm/yyyy')


obitos_tb = pd.DataFrame(columns=casost.columns, dtype=casost.dtypes.values)

for dt, df in casost.loc[casost['obito']=='SIM'].sort_values('dt_obito').groupby('dt_obito'):
    obitos_tb = obitos_tb.append(df.sort_values('nome_hash'))

obitos_tb[['nome','sexo','idade','mun_resid','telefone','dt_obito']].to_excel(writer,index=None)
writer.save()
writer.close()

In [31]:
obitos_tb

,identificacao,ibge_res_pr,ibge_atend_pr,nome,sexo,idade,mun_resid,mun_atend,laboratorio,dt_diag,dt_notificacao,dt_inicio_sintomas,15_dia_isolamento,dt_viagem_retorno,local_viagem,exposicao,data_de_internamento,local,clinico_uti,dt_alta,obito,dt_obito,muni_res_pr,hist_viagem,casos_novos,telefone,dt_atualizacao,fonte,status,eas,privado_sus,uti,dt_internamento,evolucao,dt_evolucao,temp_permanencia,cod_laboratorio,idade_original,dt_com_obito,dt_com_recuperado,idade_mais1,idade_menos1,nome_hash
291600,27696,4115200,4126256,ROSANGELA ANTUNES MACHADO DA,F,54,MARINGA,SARANDI,LACEN,2020-03-27,2020-03-27,2020-03-14,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,SIM,2020-03-25,NaN,NaN,NaN,NaN,NaT,,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,23,54,NaT,NaT,55,53,ROSANGELAANTUNESMACHADODA
274981,4062,4115200,4115200,BELCHIOR MARTINS,M,84,MARINGA,MARINGA,LACEN,2020-03-27,2020-03-27,2020-03-15,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,SIM,2020-03-26,NaN,NaN,NaN,NaN,NaT,,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,23,84,NaT,NaT,85,83,BELCHIORMARTINS
69690,24247,4104808,4104808,NEI SEUTER MARTINS,M,66,CASCAVEL,CASCAVEL,LACEN,2020-03-29,2020-03-29,2020-03-14,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,SIM,2020-03-30,NaN,NaN,NaN,NaN,NaT,,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,23,66,NaT,NaT,67,65,NEISEUTERMARTINS
52491,11387,4104303,4104303,GABRIEL GOLIN,M,72,CAMPO MOURAO,CAMPO MOURAO,LACEN,2020-04-02,2020-04-02,2020-03-18,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,SIM,2020-03-31,NaN,NaN,NaN,NaN,NaT,,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,23,72,NaT,NaT,73,71,GABRIELGOLIN
112214,179699,4106902,4106902,DENISE STANKEVICZ CORREA,F,40,CURITIBA,CURITIBA,HSNG,2020-04-03,2020-08-27,NaT,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,SIM,2020-04-02,NaN,NaN,NaN,NaN,NaT,,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,43,40,2020-08-27,NaT,41,39,DENISESTANKEVICZCORREA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216786,485807,4109807,4109807,JOSE SHIROMA,M,82,IBIPORA,IBIPORA,COVID-19 BIOLOGIA MOLECULAR,2020-12-23,2020-12-30,2020-12-20,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,SIM,2021-01-05,NaN,NaN,NaN,NaN,NaT,,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,130,82,2021-01-05,NaT,83,81,JOSESHIROMA
13323,501235,4101507,4101507,MARIA ELIETE DE LIMA,F,61,ARAPONGAS,ARAPONGAS,COVID-19 BIOLOGIA MOLECULAR,2021-01-04,2021-01-05,2020-12-24,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,SIM,2021-01-05,NaN,NaN,NaN,NaN,NaT,,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,130,61,2021-01-05,NaT,62,60,MARIAELIETEDELIMA
175377,476328,4107603,4107603,MARIA RIBEIRO DE LIMA,F,70,FAXINAL,FAXINAL,COVID-19 BIOLOGIA MOLECULAR,2020-12-27,2020-12-27,2020-12-16,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,SIM,2021-01-05,NaN,NaN,NaN,NaN,NaT,,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,130,70,2021-01-05,NaT,71,69,MARIARIBEIRODELIMA
323093,501970,4118402,4126256,RICARDO AUDI,M,66,PARANAVAI,SARANDI,COVID-19 BIOLOGIA MOLECULAR,2020-12-21,2021-01-05,2020-11-30,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaT,SIM,2021-01-05,NaN,NaN,NaN,NaN,NaT,,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,130,66,2021-01-05,NaT,67,65,RICARDOAUDI
